<a href="https://colab.research.google.com/github/adarshkuthuru/311-service-Data-Analysis-Project/blob/main/DHL_MVP_standalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all the libraries

In [39]:
import pandas as pd
import math
import numpy as np

#to import files from google drive
from pydrive.auth import GoogleAuth #!pip install -U -q PyDrive
from pydrive.drive import GoogleDrive
from google.colab import auth #!pip install google-colab
from oauth2client.client import GoogleCredentials
from google.colab import drive

Upload file on Google Drive and authorize

In [40]:
#drive.mount('/content/gdrive') #comment out after running once
xls=pd.ExcelFile('/content/gdrive/My Drive/Colab Notebooks/_Form.xlsx') #,sheet_name='1. Data Model Raw Attr') #, skiprows = range(2,150))#, usecols = "B:L")
df = xls.parse('1. Data Model Raw Attr', skiprows=3, index_col=None, na_values=['NA'])

Print and check the dataframe

In [41]:
df.head()
#print(df.shape)

,Unnamed: 0,S. No.,Section,Variable Name,Variable Decsription,Comments,"Source (Where to get data from, Query, Exact NetO2 Screen)",NetO2 Screen,NetO2 Attribute Name,Query Attribute table,Query Attribute name,Loan 1,Loan 2,Unnamed: 13
0,NaN,NaN,NaN,CLNT_LOAN_APLN_NBR,Loan Application Number,NaN,NaN,NaN,NaN,NaN,NaN,1008396376,NaN,NaN
1,NaN,1.0,1.0,Application Decline Date,The date on which loan application was declined,NaN,NetOxygen,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2.0,1.0,Declined Application Review,This variable shows if review for an applicati...,NaN,NetOxygen,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3.0,1.0,Sharepoint site updated,?,NaN,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4.0,1.0,Income decreased,Shows if income has decreased after pre-UW or not,NaN,NetOxygen,NaN,NaN,NaN,NaN,False,NaN,NaN


Drop irrelevant variables, transpose the dataframe and write csv file

In [42]:
df1=df[['Variable Name','Loan 1', 'Loan 2']]
df2= df1.set_index('Variable Name').T
df2.columns = df2.columns.str.replace("[';','.']", "") # replace special characters from column names
df2.columns = df2.columns.str.replace("[#,@,&,' ',',',(,)]", "_")  # replace special characters from column names

df2.to_csv('/content/gdrive/My Drive/Colab Notebooks/Input Data For Python.csv')
#df2.head()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, Loan 1 to Loan 2
Columns: 174 entries, CLNT_LOAN_APLN_NBR to bankruptcy_in_last_7years_dec
dtypes: datetime64[ns](1), object(173)
memory usage: 2.7+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


Section 1 to 5 code:

In [43]:
from pandas.core.arrays.sparse import dtype

#add new columns for the rule passes
#df2=df2.drop(['rule_1_1_1_pass','rule_1_1_2_pass'],axis=1)
#df2[['rule_1_1_1_pass','rule_1_1_2_pass']]=pd.DataFrame([[pd.NA,pd.NA]], index=df2.index, dtype=bool)
# df2['rule_1_1_1_pass'] = np.where(math.isnan(df2['Application Decline Date']), True, False)
# df2['rule_1_1_2_pass'] = np.where(df2['Application Decline Date']==np.nan, False, True)
#df2.head()
df3 = df2.copy()
#df3.loc[df3.Application_Decline_Date == np.nan, 'rule_1_1_1_pass'] = True
df3.head()

Variable Name,CLNT_LOAN_APLN_NBR,Application_Decline_Date,Declined_Application_Review,Sharepoint_site_updated,Income_decreased,Credit_score_decreased,Conditions_added,Omitted_debts_decreased,NO_UW_CASE__Bankruptcy_Foreclosure_Employment_,UW_Case,Extenuating_circumstances,Verifying_documentation,Credit_Score,asset_reserves,Asset_Title_Name_1003_Application,Asset_Title_Report_LOE,Credit_Report_1003_Application,Credit_Report_1003_Application_LOE,PRPRTY_ST_CDE,PRPRTY_ST_CDE_other_attributes,mortgage_meets_guidelines,SSN_consistent,Liabilities_consistent,alerts_clear,inquiries_addressed,additional_address_match,Additional_properties_in_the_last_12_months,Address_Match,First_Mtg_Eligible,ARM_ind,payment_accurate,online_calculator,Mortgage_modification,modification_docs,def_balance_CLTV,Mortgage_Statement_Exp_Date,tax_calc_proc,insurance_calc_proc,date_calc,PRPRTY_ST_CDE,...,DOB_on_Photo_ID_DOB_on_1003,Explanation_for_mismatch,Name_on_DL_Name_on_1003,Reason_for_discrepancy,Nationality,Resident_Alien_Card,Category_DL_State_ID,Type_DL_State_ID,PRPRTY_TYP_DSC,not_under_construction,not_conotel,monthly_due,NO_Trust_on_File,title_has_trust,OCC_TYP_NM,HPML_loans,Residual_income_calc,Fails_addressed,NTB_More_Than_5K,NoHighCost_being_True,NoHighPr_being_True,HPML_being_False_OR_First_lien_being_False,HPML_being_True_and_First_lien_being_True_escrowacct_exist,Compliance_ran_with5daysofApproval,income_verified_,assets_verified,Conditions_added,<date_on_which_conditions_added>,UW_review_dec,Primary_residence_dec,non_disclosed_credit_dec,priority_over_first_mortgage_lien,cosigner_or_guarantor_on_any_debt_not_disclosed_dec,outstanding_judgements_dec,delinquent_or_default_on_fed_debt_dec,personal_liability_in_a_lawsuit_dec,conveyed_title_in_lieu_of_foreclosure_in_past_7years_dec,property_sold_for_less_than_outstanding_mortgage_due_dec,foreclosure_in_last_7years_dec,bankruptcy_in_last_7years_dec
Loan 1,1008396376,NaN,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,707,True,True,True,Yes,Yes,FL,NaN,Ok,Yes,Yes,Yes,None noted,None noted,False,Yes,Yes,NaN,True,True,NaN,YES,YES,YES,Assumed Yes,Assumed Yes,2021-05-24,FL,...,YES,True,YES,True,True,NaN,YES,YES,A - Single Family,NaN,NaN,NaN,NaN,NaN,Owner Occupied,False,YES,YES,Will review at final underwrite,No,"Yes, MFJ, Family size of 4 and 1,955 sq ft on AVM",Y,True,True,True,True,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES
Loan 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#type(df2['rule_1_1_1_pass'].iloc[1])

pandas._libs.missing.NAType

In [ ]:
#df3.head()

NameError: ignored

In [49]:
import copy
headers = columns=list(df2)
new_df = pd.DataFrame()

for index, row in df2.iterrows():
    new_row = copy.deepcopy(row)
    new_dict= new_row.to_dict()
    if pd.isna(row['Application_Decline_Date'])==True:
      new_dict['rule_1_1_1_pass'] = 'True'
      #new_dict['rule_1_1_2_pass']= False
    else:
      #new_dict['rule_1_1_1_pass'] = False
      new_dict['rule_1_1_2_pass']='True'
    print(new_dict)
    new_df=new_df.append(new_dict, ignore_index=True)

new_df.head()
    

{'CLNT_LOAN_APLN_NBR': 1008396376, 'Application_Decline_Date': nan, 'Declined_Application_Review': nan, 'Sharepoint_site_updated': nan, 'Income_decreased': False, 'Credit_score_decreased': False, 'Conditions_added': 'YES', 'Omitted_debts_decreased': False, 'NO_UW_CASE__Bankruptcy_Foreclosure_Employment_': nan, 'UW_Case': nan, 'Extenuating_circumstances': nan, 'Verifying_documentation': nan, 'Credit_Score': 707, 'asset_reserves': True, 'Asset_Title_Name_1003_Application': True, 'Asset_Title_Report_LOE': True, 'Credit_Report_1003_Application': 'Yes', 'Credit_Report_1003_Application_LOE': 'Yes', 'PRPRTY_ST_CDE': 'FL', 'PRPRTY_ST_CDE_other_attributes': nan, 'mortgage_meets_guidelines': 'Ok', 'SSN_consistent': 'Yes', 'Liabilities_consistent': 'Yes', 'alerts_clear': 'Yes', 'inquiries_addressed': 'None noted', 'additional_address_match': 'None noted', 'Additional_properties_in_the_last_12_months': False, 'Address_Match': 'Yes', 'First_Mtg_Eligible': 'Yes', 'ARM_ind': nan, 'payment_accurate': 

,CLNT_LOAN_APLN_NBR,Application_Decline_Date,Declined_Application_Review,Sharepoint_site_updated,Income_decreased,Credit_score_decreased,Conditions_added,Omitted_debts_decreased,NO_UW_CASE__Bankruptcy_Foreclosure_Employment_,UW_Case,Extenuating_circumstances,Verifying_documentation,Credit_Score,asset_reserves,Asset_Title_Name_1003_Application,Asset_Title_Report_LOE,Credit_Report_1003_Application,Credit_Report_1003_Application_LOE,PRPRTY_ST_CDE,PRPRTY_ST_CDE_other_attributes,mortgage_meets_guidelines,SSN_consistent,Liabilities_consistent,alerts_clear,inquiries_addressed,additional_address_match,Additional_properties_in_the_last_12_months,Address_Match,First_Mtg_Eligible,ARM_ind,payment_accurate,online_calculator,Mortgage_modification,modification_docs,def_balance_CLTV,Mortgage_Statement_Exp_Date,tax_calc_proc,insurance_calc_proc,date_calc,Address_match,...,DOB_on_Photo_ID_DOB_on_1003,Explanation_for_mismatch,Name_on_DL_Name_on_1003,Reason_for_discrepancy,Nationality,Resident_Alien_Card,Category_DL_State_ID,Type_DL_State_ID,PRPRTY_TYP_DSC,not_under_construction,not_conotel,monthly_due,NO_Trust_on_File,title_has_trust,OCC_TYP_NM,HPML_loans,Residual_income_calc,Fails_addressed,NTB_More_Than_5K,NoHighCost_being_True,NoHighPr_being_True,HPML_being_False_OR_First_lien_being_False,HPML_being_True_and_First_lien_being_True_escrowacct_exist,Compliance_ran_with5daysofApproval,income_verified_,assets_verified,<date_on_which_conditions_added>,UW_review_dec,Primary_residence_dec,non_disclosed_credit_dec,priority_over_first_mortgage_lien,cosigner_or_guarantor_on_any_debt_not_disclosed_dec,outstanding_judgements_dec,delinquent_or_default_on_fed_debt_dec,personal_liability_in_a_lawsuit_dec,conveyed_title_in_lieu_of_foreclosure_in_past_7years_dec,property_sold_for_less_than_outstanding_mortgage_due_dec,foreclosure_in_last_7years_dec,bankruptcy_in_last_7years_dec,rule_1_1_1_pass
0,1008396376.0,NaN,NaN,NaN,0.0,0.0,YES,0.0,NaN,NaN,NaN,NaN,707.0,1.0,1.0,1.0,Yes,Yes,FL,NaN,Ok,Yes,Yes,Yes,None noted,None noted,0.0,Yes,Yes,NaN,1.0,1.0,NaN,YES,YES,YES,Assumed Yes,Assumed Yes,2021-05-24,YES,...,YES,1.0,YES,1.0,1.0,NaN,YES,YES,A - Single Family,NaN,NaN,NaN,NaN,NaN,Owner Occupied,0.0,YES,YES,Will review at final underwrite,No,"Yes, MFJ, Family size of 4 and 1,955 sq ft on AVM",Y,1.0,1.0,1.0,1.0,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,True
1,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
#df2.loc[df2["Application Decline Date"]=='NaN', "rule_1_1_1_pass"] = True

for i in range(1,2): #Loan-1: current focus

  #Rule 1.1:
  if math.isnan(df3['Application_Decline_Date'][i])==True:
  #if df3['Application Decline Date'][i]==np.nan:
  
    #df3['rule_1_1_1_pass'][i]=1
    df3['rule_1_1_1_pass'].iloc[1]='True'
  else:
    #df3['rule_1_1_2_pass'][i]=1
    df3['rule_1_1_2_pass'].iloc[1]='True'

df3.head()

#print(df2['rule_1_1_2_pass'][i])

#Subrule 1.1.2:
# if rule_1_1_2_pass==True:





/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Variable Name,CLNT_LOAN_APLN_NBR,Application_Decline_Date,Declined_Application_Review,Sharepoint_site_updated,Income_decreased,Credit_score_decreased,Conditions_added,Omitted_debts_decreased,NO_UW_CASE__Bankruptcy_Foreclosure_Employment_,UW_Case,Extenuating_circumstances,Verifying_documentation,Credit_Score,asset_reserves,Asset_Title_Name_1003_Application,Asset_Title_Report_LOE,Credit_Report_1003_Application,Credit_Report_1003_Application_LOE,PRPRTY_ST_CDE,PRPRTY_ST_CDE_other_attributes,mortgage_meets_guidelines,SSN_consistent,Liabilities_consistent,alerts_clear,inquiries_addressed,additional_address_match,Additional_properties_in_the_last_12_months,Address_Match,First_Mtg_Eligible,ARM_ind,payment_accurate,online_calculator,Mortgage_modification,modification_docs,def_balance_CLTV,Mortgage_Statement_Exp_Date,tax_calc_proc,insurance_calc_proc,date_calc,PRPRTY_ST_CDE,...,Name_on_DL_Name_on_1003,Reason_for_discrepancy,Nationality,Resident_Alien_Card,Category_DL_State_ID,Type_DL_State_ID,PRPRTY_TYP_DSC,not_under_construction,not_conotel,monthly_due,NO_Trust_on_File,title_has_trust,OCC_TYP_NM,HPML_loans,Residual_income_calc,Fails_addressed,NTB_More_Than_5K,NoHighCost_being_True,NoHighPr_being_True,HPML_being_False_OR_First_lien_being_False,HPML_being_True_and_First_lien_being_True_escrowacct_exist,Compliance_ran_with5daysofApproval,income_verified_,assets_verified,Conditions_added,<date_on_which_conditions_added>,UW_review_dec,Primary_residence_dec,non_disclosed_credit_dec,priority_over_first_mortgage_lien,cosigner_or_guarantor_on_any_debt_not_disclosed_dec,outstanding_judgements_dec,delinquent_or_default_on_fed_debt_dec,personal_liability_in_a_lawsuit_dec,conveyed_title_in_lieu_of_foreclosure_in_past_7years_dec,property_sold_for_less_than_outstanding_mortgage_due_dec,foreclosure_in_last_7years_dec,bankruptcy_in_last_7years_dec,rule_1_1_1_pass,rule_1_1_2_pass
Loan 1,1008396376,NaN,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,707,True,True,True,Yes,Yes,FL,NaN,Ok,Yes,Yes,Yes,None noted,None noted,False,Yes,Yes,NaN,True,True,NaN,YES,YES,YES,Assumed Yes,Assumed Yes,2021-05-24,FL,...,YES,True,True,NaN,YES,YES,A - Single Family,NaN,NaN,NaN,NaN,NaN,Owner Occupied,False,YES,YES,Will review at final underwrite,No,"Yes, MFJ, Family size of 4 and 1,955 sq ft on AVM",Y,True,True,True,True,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,<NA>,<NA>
Loan 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>


In [ ]:
df3['rule_1_1_1_pass'].iloc[1]=True
df3.head()
#type(df2['Application Decline Date'].iloc[1])
#type(math.isnan(df2['Application Decline Date'].iloc[1]))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Variable Name,CLNT_LOAN_APLN_NBR,Application Decline Date,Declined Application Review,Sharepoint site updated,Income decreased,Credit score decreased,Conditions added,Omitted debts decreased,"NO UW CASE (Bankruptcy, Foreclosure, Employment)",UW Case,Extenuating circumstances,Verifying documentation,Credit Score,asset_reserves,"Asset Title Name, 1003 Application",Asset Title Report LOE,"Credit Report, 1003 Application","Credit Report, 1003 Application, LOE",PRPRTY_ST_CDE,PRPRTY_ST_CDE; other attributes,mortgage_meets_guidelines,SSN_consistent,Liabilities_consistent,alerts_clear,inquiries_addressed,additional_address_match,Additional properties in the last 12 months,Address Match..,First Mtg Eligible,ARM_ind,payment_accurate,online calculator,Mortgage modification,modification docs,def balance CLTV,Mortgage Statement Exp Date,tax_calc_proc,insurance_calc_proc,date_calc,PRPRTY_ST_CDE,...,"Name on DL, Name on 1003",Reason for discrepancy,Nationality,Resident Alien Card,Category_DL_State_ID,Type_DL_State_ID,PRPRTY_TYP_DSC,not_under_construction,not_conotel,monthly_due,NO Trust on File,title_has_trust,OCC_TYP_NM,HPML loans,Residual income calc,Fails addressed,NTB_More_Than_5K,NoHighCost_being_True,NoHighPr_being_True,HPML_being_False_OR_First_lien_being_False,HPML_being_True_and_First_lien_being_True_escrowacct_exist,Compliance_ran_with5daysofApproval,income_verified,assets_verified,Conditions added,<date on which conditions added>,UW_review_dec,Primary_residence_dec,non_disclosed_credit_dec,priority_over_first_mortgage_lien,cosigner_or_guarantor_on_any_debt_not_disclosed_dec,outstanding_judgements_dec,delinquent_or_default_on_fed_debt_dec,personal_liability_in_a_lawsuit_dec,conveyed_title_in_lieu_of_foreclosure_in_past_7years_dec,property_sold_for_less_than_outstanding_mortgage_due_dec,foreclosure_in_last_7years_dec,bankruptcy_in_last_7years_dec,rule_1_1_1_pass,rule_1_1_2_pass
Loan 1,1008396376,NaN,NaN,NaN,False,False,False,False,NaN,NaN,NaN,NaN,707,True,True,True,Yes,Yes,FL,NaN,Ok,Yes,Yes,Yes,None noted,None noted,False,Yes,Yes,NaN,True,True,NaN,YES,YES,YES,Assumed Yes,Assumed Yes,2021-05-24,FL,...,YES,True,True,NaN,YES,YES,A - Single Family,NaN,NaN,NaN,NaN,NaN,Owner Occupied,False,YES,YES,Will review at final underwrite,No,"Yes, MFJ, Family size of 4 and 1,955 sq ft on AVM",Y,True,True,True,True,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,False,False
Loan 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
